# Imports

In [ ]:
import csv
import subprocess
import json

# Config

In [ ]:
MODEL = "qwen3:8b"   # name of your local Ollama model
INPUT_CSV = "input_words.csv"
OUTPUT_CSV = "ollama_associations.csv"

# SWOW-style prompt template
PROMPT_TEMPLATE = """
You will perform a free word association task, similar to the Small World of Words (SWOW) study.

Task:
I will give you a cue word. Your job is to produce up to three single-word associations:
A1 = strongest association
A2 = second association
A3 = third association

Guidelines (VERY IMPORTANT):
- Output must be exactly one line with four fields separated by semicolons (;).
- Format: [cue];[A1];[A2];[A3]
- Each of A1, A2, A3 must be exactly one word (no spaces).
- If you cannot provide A2 or A3, use the exact placeholder: No more responses
- Do NOT output any other text, explanation, or extra punctuation.
- Do NOT repeat the cue as an association.
- Example valid outputs:
  dog;bark;leash;pet
  cat;meow;No more responses;No more responses

Now process the following cue word:
{cue}
"""

# Functions

In [ ]:
# ----------------------------------------------------------
# FUNCTION TO QUERY OLLAMA
# ----------------------------------------------------------
def ask_ollama(model: str, prompt: str) -> str:
    """Send a prompt to a local Ollama model and return the output text."""
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt,
        capture_output=True,
        text=True
    )
    return result.stdout.strip()

# ----------------------------------------------------------
# LOAD INPUT WORDS
# ----------------------------------------------------------
def load_cue_words(path: str):
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        return [row[0] for row in reader]  # first column only

# ----------------------------------------------------------
# SAVE OUTPUT
# ----------------------------------------------------------
def save_results(path: str, rows):
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["model", "cue", "A1", "A2", "A3"])
        writer.writerows(rows)

# Pipeline

In [ ]:
cues = load_cue_words(INPUT_CSV)
results = []

for cue in cues:
    prompt = PROMPT_TEMPLATE.format(cue=cue)
    response = ask_ollama(MODEL, prompt)

    # Expected format: "cue;answer1;answer2;answer3"
    parts = response.split(";")

    if len(parts) != 4:
        print(f"Warning: Unexpected response format for '{cue}': {response}")
        continue

    cue_out, a1, a2, a3 = parts
    results.append([MODEL, cue_out, a1, a2, a3])

save_results(OUTPUT_CSV, results)
print(f"Done! Saved to {OUTPUT_CSV}")

In [ ]:
def extract_unique_values(input_csv, output_csv, column_name, delimiter=","):
    """
    Extracts all distinct values from a given column in a CSV file and writes them
    into a new CSV file (one value per line).
    
    Args:
        input_csv (str): Path to the input CSV file.
        output_csv (str): Path to the output CSV file.
        column_name (str): Name of the column from which to collect unique values.
        delimiter (str): CSV delimiter (default=",").
    
    Returns:
        list: A list of unique values.
    """
    unique_values = set()

    # Read input CSV
    with open(input_csv, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter=delimiter)
        if column_name not in reader.fieldnames:
            raise ValueError(f"Column '{column_name}' not found in CSV.")
        
        for row in reader:
            value = row[column_name].strip()
            if value:
                unique_values.add(value)

    # Sort for consistent output
    unique_list = sorted(unique_values)

    # Write output CSV
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=delimiter)
        writer.writerow([column_name])  # header
        for val in unique_list:
            writer.writerow([val])

    return unique_list

extract_unique_values("C:\Users\peers\Downloads\SWOW-EN18\SWOW-EN.complete.20180827.csv", "cues.csv", "cue", ",")